In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# define the log-in page and searching page
log_url = "https://gs1lv.org/login_to_gepir"
search_url = "https://gs1lv.org/gepirui/search_by_gtin"

Using the console, we found that we can log in "gs1lv.org" by posting `multipart/form-data` which contains username, password, and submit information.

multipart/form-data:
```------WebKitFormBoundarywfB7ajh0Y8kouU5V
Content-Disposition: form-data; name="email"

bems@pp.u-tokyo.ac.jp
------WebKitFormBoundarywfB7ajh0Y8kouU5V
Content-Disposition: form-data; name="password"

95a49208a
------WebKitFormBoundarywfB7ajh0Y8kouU5V
Content-Disposition: form-data; name="submit"

Pieslēgties
------WebKitFormBoundarywfB7ajh0Y8kouU5V--```

In [3]:
# create a requests session
s = requests.Session()

In [4]:
# log in by posting multipart/form-data
# cookies are stored in requests session automatically
# note that using the format like {'email': 'bems@pp.u-tokyo.ac.jp'} would cause error
s.post(log_url, files={'email': (None, 'bems@pp.u-tokyo.ac.jp'), 
                        'password': (None, '95a49208a'), 
                        'submit': (None, 'Pieslēgties')})

<Response [200]>

In [5]:
GTVN = ['4750346000160', '4750334000714', '4750327021900']

In [6]:
def GTVN_searcher(GTVN):
    
    data = {}
    
    for number in GTVN:
        data[number] = {}
        for info in ['ownership', 'information']:
            data[number][info] = {}
            # post form data for searching through requests session
            r = s.post(search_url, data={'requestedGTIN': number, 'requestTradeItem': info})
            # extract the information from content by BeautifulSoup
            soup = BeautifulSoup(r.content, "lxml")
            tds = soup.findAll('table')[0].findAll('td')
            for i in range(len(tds)//2):
                data[number][info][tds[2*i].text.split(':')[0]] = tds[2*i+1].text
    return data

In [7]:
%%timeit
global data
data = GTVN_searcher(GTVN)

1 loop, best of 3: 12.4 s per loop


In [8]:
data

{'4750327021900': {'information': {'Nosaukums': 'Forele speciālā sālījumā ',
   'Numurs': '04750327021900',
   'Preču zīme': 'Zamelsons',
   'Pārdošanas vienība': 'BASE_UNIT_OR_EACH'},
  'ownership': {'GVN': '4750327000004',
   'Provaidera GVN': '4750475000000',
   'Pēdējās izmaiņas': '2012-01-04',
   'Uzņēmuma numurs': '4750327',
   'Uzņēmums': 'ZAMELSONS SIA'}},
 '4750334000714': {'information': {'Nosaukums': 'Garšviela hmeli suneli\r\n\r\n\r\n\r\n',
   'Numurs': '04750334000714',
   'Preču zīme': 'Valežs',
   'Pārdošanas vienība': 'BASE_UNIT_OR_EACH'},
  'ownership': {'GVN': '4750334000004',
   'Provaidera GVN': '4750475000000',
   'Pēdējās izmaiņas': '1998-01-28',
   'Uzņēmuma numurs': '4750334',
   'Uzņēmums': 'VALEŽS SIA RKF'}},
 '4750346000160': {'information': {'Nosaukums': ' Talsu kefīrs 2,5% bioloģiskā lauksaimniecība',
   'Numurs': '04750346000160',
   'Preču zīme': 'N/A',
   'Pārdošanas vienība': 'BASE_UNIT_OR_EACH'},
  'ownership': {'GVN': '4750346000009',
   'Provaidera G